In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.18-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
from drive.ML_Study import * # 让driver连接云盘的ML_Study

In [0]:
# For console
# 防断连：f12 复制到console里边运行 
function ClickConnect(){
  console.log("Working"); 
  document
    .querySelector("#top-toolbar > colab-connect-button")
    .shadowRoot
    .querySelector("#connect")
    .click()
}
 
setInterval(ClickConnect,60000)

In [5]:
pip install transformers

     |████████████████████████████████| 552kB 1.4MB/s 
     |████████████████████████████████| 870kB 52.2MB/s 
     |████████████████████████████████| 1.0MB 49.3MB/s 
     |████████████████████████████████| 3.7MB 51.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=be8241f50dbfb100c80bd5b45a28c3dac805bb9b905ea7837b564565f692b96b
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import json
import numpy as np
from tqdm import tqdm
import os
import tensorflow as tf
from transformers import *
import tensorflow_addons as tfa

In [0]:
# input最大长度
maxlen=128

In [8]:
print('Lodaing BERT')
# 最好使用自己下载的文件，否则可能会因为连不上下载文件的服务器而报错
# 而且不是bertGithub的包，使用的是transformers提供的，需要去官方文档那里下载
pretrainedModel_path = 'drive/ML_Study/bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(os.path.join(pretrainedModel_path, 'vocab.txt'))
config = BertConfig.from_json_file(os.path.join(pretrainedModel_path, 'config.json'))
print('==========END==========')

Lodaing BERT


Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


==========END==========


In [0]:
# 生成新的SPO
def load_data(path):
    D = []
    with open(path, encoding='utf-8') as f:
        for l in f.readlines(): 
            l = json.loads(l)
            D.append({
                'text': l['text'],
                'spo_list': [
                    (spo['subject'], spo['predicate'], spo['object']['@value'])
                    for spo in l['spo_list']
                ]
            })
    return D

# 得到s,p对应值的初始下标
def search(pattern, sequence):
    n = len(pattern)
    for i in range(len(sequence)):
        if sequence[i:i + n] == pattern:
            return i
    return -1

# 寻找特殊符号的位置
def search_special_label(pattern, sequence):
    rt = []
    for i, sid in enumerate(sequence):
        if sid == pattern:
            rt.append(i)
    return rt

In [10]:
print('Loading Data')
# train_data = load_data('drive/ML_Study/Baidu_ERE_Data/sample_data.json')
train_data = load_data('drive/ML_Study/Baidu_ERE_Data/train_data.json')
# valid_data = load_data('drive/ML_Study/B4K_Data/dev_data.json')
# test_data = load_data('./data/example.test')
print('=========END==========')

Loading Data
=========END==========


In [0]:
# category mapping
def get_mapping(mapping_path):
    predicate2id, id2predicate = {}, {}
    with open(mapping_path, encoding='utf-8') as f:
        for l in f.readlines():
            l = json.loads(l)
            if l['predicate'] not in predicate2id:
                id2predicate[len(predicate2id)] = l['predicate']
                predicate2id[l['predicate']] = len(predicate2id)
    return predicate2id, id2predicate
    
mapping_path = 'drive/ML_Study/Baidu_ERE_Data/schema.json'

predicate2id, id2predicate = get_mapping(mapping_path)

In [0]:
# get BIO-relation label mapping
# get predicate mapping
special_labels = ['[PAD]','[CLS]','[SEP]','[UNK]']
special_labels_token = [0,101,102,100]
# All label
labels = []
for l in predicate2id.keys():
    l = 'B-Sub-'+l
    labels.append(l)
for l in predicate2id.keys():
    l = 'B-Obj-'+l
    labels.append(l)
labels.append('I')
labels.append('O')
for l in special_labels:
    labels.append(l)
    
# Bilabel Mapping
id2BIO = {}
BIO2id = {}
for i,label in enumerate(labels):
    id2BIO[i] = label
    BIO2id[label] = i

In [0]:
# 这个func应该没用了

# 0 1 形式label转为 BIO 形式 
# 由于可能一个实体可能既是SUB又是OBJ 而且可能对应多种关系 故采用list的格式存储
def visualized_label(label,maxlen=128):
    visual_label =[]
    for i in range(maxlen):
        a=[]
        visual_label.append(a)
    text_p, category_p= np.where(label>0.5)
    for p in zip(text_p, category_p):
        visual_label[p[0]].append(id2BIO[p[1]])
    return visual_label

# print(visualized_abel(label))

In [0]:
# spo 转为 X 和 0 1 形式的 label Y
# label: dimensionX->sequencePosition, dimensionY->CategoryPosition
def get_input(data,tokenizer,predicate2id,maxlen=128):
    special_labels_token = [0,101,102,100]
    token_ids, segment_ids, position_ids= [], [], []
    spo_labels = []
    for d in data:
        # get X 
        token_id, segment_id, position_id = tokenizer.encode_plus(d['text'], max_length=maxlen,pad_to_max_length=True).values()
        token_ids.append(token_id)
        segment_ids.append(segment_id)
        position_ids.append(position_id)
        # get Y
        # 102 = 2R + 4 + 2 
        label = np.zeros((102,maxlen)) # len(labels), maxlen
        label[97,:] = 1 # 先设置所有标签都是 O
        for s, p, o in d['spo_list']:
            s = tokenizer.encode(s)[1:-1]# 101 是 CLS 102 是ESP 这里只是为了得到s内部值的ID
            p_i = predicate2id[p] # 对category进行映射得到对应的ID
            o = tokenizer.encode(o)[1:-1] 
            s_i = search(s, token_id) # 对text进行映射
            o_i = search(o, token_id)
            # 96:'O', 97:'I'
            # Sub
            label[p_i,s_i] = 1
            label[97,s_i] = 0
            for i in range(1,len(s)):
                s_I_i = s_i + i
                label[96,s_I_i] = 1
                label[97,s_I_i] = 0
            # Obj
            label[48+p_i,o_i] = 1
            label[97,o_i] = 0
            for i in range(1,len(o)):
                o_I_i = o_i + i
                label[96,o_I_i] = 1
                label[97,o_I_i] = 0
        # specail label
        # 98:'[PAD]'......
        for i, sp in enumerate(special_labels_token):
            sp_is = search_special_label(sp, token_id)
            for sp_i in sp_is:
                label[98+i,sp_i] = 1
                label[97,sp_i] = 0
                
        spo_labels.append(label.T)

    return [token_ids, position_ids, segment_ids, spo_labels]

In [17]:
print('Contructing Data')
#这里就可能很大了 可能要先保存为tfrecords再读取 不然不够内存
train_xy = get_input(train_data,tokenizer,predicate2id,maxlen=128)
print('==========END==========')

Contructing Data
==========END==========


In [0]:
tfr_data_path = r'drive/ML_Study/Baidu_ERE_Data/TFrecord_data/train.tfrecords'

In [41]:
# tfrecord 未完成
# 改：label 是会报错的 应为二维numpy转不了 example 需要处理
with tf.io.TFRecordWriter(tfr_data_path) as writer:
    for token_id, position_id, segment_id, spo_label in zip(train_xy[0],train_xy[1],train_xy[2],train_xy[3]):
        feature ={
            'input_ids' : tf.train.Feature(int64_list=tf.train.Int64List(value=[token_id])),
            'attention_mask' : tf.train.Feature(int64_list=tf.train.Int64List(value=[position_id])),
            'token_type_ids' : tf.train.Feature(int64_list=tf.train.Int64List(value=[segment_id])),
            'label' : tf.train.Feature(float_list=tf.train.FloatList(value=[spo_label]))
        }
        example = tf.train.Example(features=tf.train.Features(feature=feature))
        writer.write(example.SerializeToString())  
    writer.close()
print('==========END==========')

TypeError: ignored

In [0]:
class SPO_Model(TFBertPreTrainedModel):
    # @property
    # def dummy_inputs(self):
        # """ Dummy inputs to build the network.

        # Returns:
        #     tf.Tensor with dummy inputs
        # """
        # return {"input_ids": tf.constant(DUMMY_INPUTS)}

    def __init__(self, config, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.bert = TFBertMainLayer(config, name="bert")
        self.dropout = tf.keras.layers.Dropout(config.hidden_dropout_prob)
        self.classifier = tf.keras.layers.Dense(102,activation=tf.nn.sigmoid,kernel_initializer=tf.random_normal_initializer(mean=0.0, stddev=0.05))
    def call(self, inputs, **kwargs):
        bert_outputs = self.bert(inputs, **kwargs)
        sequence_output = bert_outputs[0]
        x = self.dropout(sequence_output, training=kwargs.get("training", False))
        x = self.classifier(x)
        # x = tf.keras.layers.Lambda(lambda x: x**(1/0.95))(x) #对BinaryEntropy的放大处理，假设95%的数据都是正确的(排除人工标注错误)
        # x = tf.transpose(x,perm=[0,2,1])

        # 改： 可以试试加BiLSTM和CRF，或者只加CRF，数据量一大应该有用
        # 改： dense的 kernel_initializer 可能要改
        return x

In [0]:
def myloss(y_true, y_pred):
	return tf.reduce_sum(tf.keras.losses.binary_crossentropy(y_true,y_pred))

In [0]:
spo_model = SPO_Model.from_pretrained(pretrainedModel_path,config=config)

spo_model.compile(
           optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
           loss=myloss, 
           metrics=[tf.keras.metrics.BinaryAccuracy()]) # accuracy没有好的 随便选了一个

In [19]:
spo_model.summary()

Model: "spo__model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  102267648 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  78438     
Total params: 102,346,086
Trainable params: 102,346,086
Non-trainable params: 0
_________________________________________________________________


In [20]:
spo_model.fit(x={'input_ids':train_xy[0], 'attention_mask':train_xy[1], 'token_type_ids':train_xy[2]},
          y=train_xy[3],
          epochs=20,
          validation_steps=7,
          validation_data=([train_xy[0][7000:], train_xy[1][7000:], train_xy[2][7000:]], train_xy[3][7000:]),
          batch_size=16
           )
# 下边的提示是因为我没有使用bert的pool_output

# 改：使用TFrecord后试试能不能将batch_size改大？
#   然后将epoch调大？但我是数据小才设20，如果数据多了可能会过拟合？试一下！我感觉是lossEntropy<0.5左右就合适了

Epoch 1/20
626/626 [==============================] - 346s 553ms/step - loss: 102.4228 - binary_accuracy: 0.9920 - val_loss: 31.2979 - val_binary_accuracy: 0.9982
Epoch 2/20
626/626 [==============================] - 344s 549ms/step - loss: 24.0791 - binary_accuracy: 0.9987 - val_loss: 14.8470 - val_binary_accuracy: 0.9991
Epoch 3/20
626/626 [==============================] - 344s 549ms/step - loss: 13.4938 - binary_accuracy: 0.9991 - val_loss: 9.5669 - val_binary_accuracy: 0.9992
Epoch 4/20
626/626 [==============================] - 344s 550ms/step - loss: 9.2691 - binary_accuracy: 0.9992 - val_loss: 6.9558 - val_binary_accuracy: 0.9993
Epoch 5/20
626/626 [==============================] - 344s 550ms/step - loss: 7.0137 - binary_accuracy: 0.9993 - val_loss: 5.3585 - val_binary_accuracy: 0.9994
Epoch 6/20
626/626 [==============================] - 345s 551ms/step - loss: 5.5050 - binary_accuracy: 0.9993 - val_loss: 4.2796 - val_binary_accuracy: 0.9994
Epoch 7/20
626/626 [==============

In [0]:
# spo_model.save_weights('drive/ML_Study/saved_model/weight_200331/spo_model_200331.ckpt') # 保存权重

In [0]:
m1 = SPO_Model.from_pretrained(pretrainedModel_path,config=config)

In [19]:
m1.load_weights('drive/ML_Study/saved_model/weight_200331/spo_model_200331.ckpt') # 加载已经训练的权重

In [0]:
# loss, acc = m1.evaluate(input, label)

In [32]:
# 测试
d = train_data[2]
print(d['text'])   
print(d['spo_list'])

贵州顺和建筑工程有限责任公司于2008年1月23日在贵阳市工商行政管理局登记成立
[('贵州顺和建筑工程有限责任公司', '成立日期', '2008年1月23日')]


In [0]:
t_token = tokenizer.encode(d['text'],max_length=128,pad_to_max_length=True)
input_token = tf.convert_to_tensor(t_token)[None, :]

In [34]:
# rt1 = spo_model.predict([t_token])
rt2 = m1(input_token) # 输入必须是tensor
# print(rt1[0]) 
print(rt2[0]) # 直接使用模型和加载保存的模型输出对比 小数点大约后7位开始会有些许不同

tf.Tensor(
[[5.0193185e-06 5.6911274e-05 8.4620897e-06 ... 9.9981052e-01
  3.4196801e-05 7.0086178e-05]
 [2.7969754e-05 8.3286970e-05 1.6359114e-05 ... 7.7113007e-05
  6.5531349e-05 2.2038916e-04]
 [2.7397862e-05 1.0262808e-05 7.2903558e-06 ... 1.5741525e-05
  1.2611612e-05 1.7541879e-05]
 ...
 [4.5225247e-06 3.6320348e-06 4.9338355e-06 ... 2.2255656e-06
  7.2171883e-06 3.0852075e-06]
 [4.5238753e-06 3.6251172e-06 4.8920101e-06 ... 2.2198169e-06
  7.4550248e-06 3.1191078e-06]
 [4.6151899e-06 3.7761849e-06 4.9036448e-06 ... 2.2287813e-06
  7.4868722e-06 3.2095372e-06]], shape=(128, 102), dtype=float32)


In [35]:
# 01label -> SPO
label = rt2[0]

token_id, segment_id, position_id = tokenizer.encode_plus(d['text'], max_length=maxlen,pad_to_max_length=True).values()

sub_text_p, sub_category_p = np.where(label[:,:48]>0.5) # 可以改为0.6？
obj_text_p, obj_category_p = np.where(label[:,48:96]>0.5) # obj_category_p 自动减去48!! 返回的值是从 0 -> 47!
i_text_p = sorted(np.where(label[:,96]>0.5)) # I 的对应下标
sub_info = list(zip(sub_text_p, sub_category_p))
obj_info = list(zip(obj_text_p, obj_category_p))

SPOes=[]

for sub in sub_info:
    sub_start = sub[0]
    sub_end = sub_start
    for I_i in i_text_p[0]: # 确定 sub 的长度
        if I_i == (sub_end + 1):
            sub_end += 1
    for obj in obj_info: 
        if sub[1] == (obj[1]):
            obj_start = obj[0]
            obj_end =  obj_start
            for I_i in i_text_p[0]: # 确定 obj 的长度
                if I_i == (obj_end + 1):
                    obj_end += 1
            spo = (
                tokenizer.decode(token_id[sub_start: sub_end+1]),
                id2predicate[sub[1]],
                tokenizer.decode(token_id[obj_start: obj_end+1])
                )
            SPOes.append(spo)
#y_true
print(d['text'])
print(d['spo_list'])
print('='*47)   
#y_pred   
print(SPOes)

贵州顺和建筑工程有限责任公司于2008年1月23日在贵阳市工商行政管理局登记成立
[('贵州顺和建筑工程有限责任公司', '成立日期', '2008年1月23日')]
[('贵 州 顺 和 建 筑 工 程 有 限 责 任 公 司', '成立日期', '2008 年 1 月 23 日')]


In [0]:
# label -> SPOes
def predict_SPOes(token_id, label): # zip(token_id, label_pred)
  SPOes=[]
  sub_text_p, sub_category_p = np.where(label[:,:48]>0.5) 
  obj_text_p, obj_category_p = np.where(label[:,48:96]>0.5) # obj_category_p 自动减去48!! 返回的值是从 0 -> 47!
  i_text_p = sorted(np.where(label[:,96]>0.5)) # I 的对应下标
  sub_info = list(zip(sub_text_p, sub_category_p))
  obj_info = list(zip(obj_text_p, obj_category_p))

  SPOes=[]

  for sub in sub_info:
    sub_start = sub[0]
    sub_end = sub_start
    for I_i in i_text_p[0]: # 确定 sub 的长度
        if I_i == (sub_end + 1):
            sub_end += 1
    for obj in obj_info: 
        if sub[1] == (obj[1]):
            obj_start = obj[0]
            obj_end =  obj_start
            for I_i in i_text_p[0]: # 确定 obj 的长度
                if I_i == (obj_end + 1):
                    obj_end += 1
            spo = (
                tokenizer.decode(token_id[sub_start: sub_end+1]),
                id2predicate[sub[1]],
                tokenizer.decode(token_id[obj_start: obj_end+1])
                )
            SPOes.append(spo)
  return SPOes

In [0]:
# 对test集合进行预测并输出比赛结果：
spo_M = 
token_id_test = test_xy[0]
# load model 
spo_M = SPO_Model.from_pretrained(pretrainedModel_path,config=config)
spo_M.load_weights('drive/ML_Study/saved_model/weight_200331/spo_model_200331.ckpt')
# 还有5个专门针对复杂spo的模型
# c_spo_M1 = 

# output json
f = open('dev_pred.json', 'w', encoding='utf-8') 
# test
spo_list_pred_test = []
label_pred_test = spo_M(token_id_test)

for c in zip(token_id_test, label_pred_test):
  SPOes = predict_SPOes(token_id, label)
  spo_list_pred_test.append(SPOes)


In [0]:
# 取交集用
class SPO(tuple):
    def __init__(self, spo):
        self.spox = (
            tuple(tokenizer.tokenize(spo[0])),
            spo[1],
            tuple(tokenizer.tokenize(spo[2])),
        )

    def __hash__(self):
        return self.spox.__hash__()

    def __eq__(self, spo):
        return self.spox == spo.spox

In [0]:
def evaluate(data):
    X, Y, Z = 1e-10, 1e-10, 1e-10
    f = open('dev_pred.json', 'w', encoding='utf-8') # 写入
    pbar = tqdm()
    for d in data:
        R = set([SPO(spo) for spo in extract_SPOes(d['text'])]) # pred
        T = set([SPO(spo) for spo in d['spo_list']]) # true
        X += len(R & T)
        Y += len(R)
        Z += len(T)
        f1, precision, recall = 2 * X / (Y + Z), X / Y, X / Z
        pbar.update()
        pbar.set_description('f1: %.5f, precision: %.5f, recall: %.5f' %
                             (f1, precision, recall))
        s = json.dumps(
            {
                'text': d['text'],
                'spo_list': list(T),
                'spo_list_pred': list(R),
                'new': list(R - T),
                'lack': list(T - R),
            },
            ensure_ascii=False,
            indent=4)
        f.write(s + '\n')
    pbar.close()
    f.close()
    return f1, precision, recall

In [0]:
class Evaluator(tf.keras.callbacks.Callback):
    """
    评估和保存模型
    """
    def __init__(self):
        self.best_val_f1 = 0.

    def on_epoch_end(self, epoch, logs=None):
        optimizer.apply_ema_weights()
        f1, precision, recall = evaluate(valid_data)
        if f1 >= self.best_val_f1:
            self.best_val_f1 = f1
#             train_model.save_weights('./save/best_model.weights')
        optimizer.reset_old_weights()
        print('f1: %.5f, precision: %.5f, recall: %.5f, best f1: %.5f\n' %
              (f1, precision, recall, self.best_val_f1))